In [7]:
import pandas as pd
import great_expectations as ge
from great_expectations.core.batch import RuntimeBatchRequest

# Sample DataFrame
data = {
    "Product": ["A", "B", "C", "D"],
    "Sales": [100, 200, None, 400]
}
df = pd.DataFrame(data)

context = ge.get_context()

# Add pandas datasource correctly using the FluentDatasource class
datasources = [ds["name"] for ds in context.list_datasources()]
if "pandas_datasource" not in datasources:
    context.add_datasource(
        name="pandas_datasource",
        class_name="PandasDatasource"
    )

suite_name = "sales_data_suite"

try:
    suite = context.get_expectation_suite(expectation_suite_name=suite_name)
except ge.exceptions.DataContextError:
    suite = context.create_expectation_suite(expectation_suite_name=suite_name)

batch_request = RuntimeBatchRequest(
    datasource_name="pandas_datasource",
    data_connector_name="default_runtime_data_connector_name",
    data_asset_name="sales_data_asset",
    runtime_parameters={"batch_data": df},
    batch_identifiers={"default_identifier_name": "default_identifier"},
)

validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=suite_name,
)

validator.expect_column_values_to_not_be_null("Sales")

validator.save_expectation_suite()

validation_result = validator.validate()

print("Validation Result Summary:")
print(validation_result)

print("\nRows with missing Sales:")
print(df[df["Sales"].isnull()])


DataContextError: Datasource is not a FluentDatasource